# Testing Jupyter 

In [32]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_excel("daily_bitcoin_ohlc.xlsx")

# print(df.head())

def calculate_daily_returns(df) -> pd.Series:
    """
        Calculate the daily returns from a DataFrame with a 'Close' column.

        Parameters:
            df (pd.DataFrame): DataFrame containing historical price data with a 'Close' column.

        Returns:
            pd.Series: A Series of daily return values (as decimal percentages).
    """

    daily_returns = df['close'].pct_change()
    # .pct_change() is a built in pandas method 
    # ( calculates the percentage change between the current and the prev element)
    return daily_returns

df['timestamp'] = pd.to_datetime(df['timestamp'])

df['Daily Return'] = calculate_daily_returns(df)

In [25]:
def extract_date_components(timestamp_input) -> dict:
    """
    Transform a timestamp (string in “YYYY-MM-DD HH:MM:SS” format *or* a
    datetime.datetime object) into a dictionary of calendar-based features.

    Parameters
    ----------
    timestamp_input : str | datetime.datetime
        Either the timestamp string (e.g. "2023-10-26 14:30:00")
        or an already-parsed datetime object.

    Returns
    -------
    dict
        {
            "year":               2023,
            "quarter":            4,
            "month_number":       10,
            "month_name":         "October",
            "week_of_year":       43,      # ISO-8601 week number
            "day_of_week_number": 4,       # Monday=1 … Sunday=7  (ISO)
            "day_of_week_name":   "Thursday"
        }
    """

# Checks if timestamp input is of instance/type datetime
    if isinstance(timestamp_input, datetime):
        dt = timestamp_input
    elif isinstance(timestamp_input, str):
        try:   
            dt = datetime.strptime(timestamp_input, "%Y-%m-%d %H:%M:%S")
        except ValueError as exc:          
            raise ValueError(
                "Timestamp string must match 'YYYY-MM-DD HH:MM:SS'"
            ) from exc
    else:
        raise TypeError(
            "timestamp_input must be a datetime object or a timestamp string"
        )


    features = {
         "year":               dt.year,
        "quarter":            (dt.month - 1) // 3 + 1,          # 1–4
        "month_number":       dt.month,                         # 1–12
        "month_name":         dt.strftime("%B"),                # "January"
        "week_number":         dt.isocalendar().week,         # "January"
        "day_of_week_number": dt.weekday() + 1,                    # 1=Mon … 7=Sun
        "day_of_week_name":   dt.strftime("%A"),                # "Monday"…
    }

    return features

In [26]:
test_output = extract_date_components("2023-10-26 14:30:00")
print(test_output)


{'year': 2023, 'quarter': 4, 'month_number': 10, 'month_name': 'October', 'week_number': 43, 'day_of_week_number': 4, 'day_of_week_name': 'Thursday'}


In [27]:
from datetime import datetime

test_dt = datetime(2025, 5, 26, 20, 15, 0)
print(extract_date_components(test_dt))


{'year': 2025, 'quarter': 2, 'month_number': 5, 'month_name': 'May', 'week_number': 22, 'day_of_week_number': 1, 'day_of_week_name': 'Monday'}


In [28]:
# Expecting a ValueError due to bad format
try:
    extract_date_components("2023/10/26")
except ValueError as e:
    print("Caught expected ValueError:", e)

# Expecting a TypeError due to wrong input type
try:
    extract_date_components(12345)
except TypeError as e:
    print("Caught expected TypeError:", e)


Caught expected ValueError: Timestamp string must match 'YYYY-MM-DD HH:MM:SS'
Caught expected TypeError: timestamp_input must be a datetime object or a timestamp string


In [29]:
# Apply to a DataFrame and explode the dict into columns
df_features = df['timestamp'].apply(extract_date_components)

# Merge with the original DataFrame
df = pd.concat([df, df_features], axis=1)

# Preview the results
df.head()


,timestamp,open,high,low,close,Daily Return,timestamp
0,2018-02-09 00:00:00+00:00,7611.61,7611.61,7611.61,7611.61,NaN,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
1,2018-02-10 00:00:00+00:00,8208.57,8674.76,7847.14,8672.57,0.139387,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
2,2018-02-11 00:00:00+00:00,8659.92,9088.97,8283.43,8590.21,-0.009497,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
3,2018-02-12 00:00:00+00:00,8583.38,8583.38,7890.82,8064.69,-0.061177,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
4,2018-02-13 00:00:00+00:00,8105.98,8920.31,8105.98,8845.22,0.096784,"{'year': 2018, 'quarter': 1, 'month_number': 2..."


------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                Volatility metrics

In [33]:
def calculate_daily_returns(df) -> pd.Series:
    """
        Calculate the daily returns from a DataFrame with a 'Close' column.

        Parameters:
            df (pd.DataFrame): DataFrame containing historical price data with a 'Close' column.

        Returns:
            pd.Series: A Series of daily return values (as decimal percentages).
    """

    daily_returns = df['close'].pct_change()
    # .pct_change() is a built in pandas method 
    # ( calculates the percentage change between the current and the prev element)
    return daily_returns

df['timestamp'] = pd.to_datetime(df['timestamp'])

df['Daily Return'] = calculate_daily_returns(df)

In [34]:
def get_return_std_dev(return_series):
    """
    Calculates the standard deviation of returns from a Series of daily returns.    

    Parameters:
        return_series (pd.Series): Series containing daily returns.

    Returns:
        float: The standard deviation of returns, rounded to 4 decimal places.

    Example usage:
        std_dev = get_return_std_dev(daily_returns)
    """
    return round(return_series.std(), 4)

In [37]:
def get_rolling_volatility(return_series, window):
    """
    Calculates the rolling standard deviation of returns over a specified window.

    Parameters:
        return_series (pd.Series): Series containing daily returns.
        window (int): The number of days to calculate the rolling standard deviation over.
    
    Returns:
        pd.Series: A Series containing the rolling standard deviation of returns.

    Example usage:
        rolling_volatility = get_rolling_volatility(daily_returns, 20)
    """
    return return_series.rolling(window).std()  


In [21]:
return_series = calculate_daily_returns(df)

In [ ]:
print("STD: ", get_return_std_dev(return_series))

rolling volatility:  0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
2647    0.022218
2648    0.022318
2649    0.023076
2650    0.022436
2651    0.021309
Name: close, Length: 2652, dtype: float64


Your standard deviation of 0.0339 (or 3.39%) for Bitcoin's daily returns is a measure of volatility. Here's what this means:

1. Daily Price Movement Context
On any given day, Bitcoin's price typically moves by about 3.39% from the previous day's closing price
This represents HIGH volatility compared to traditional assets
For reference: The S&P 500 typically has daily volatility of around 1.2%

2. Statistical Interpretation (68-95-99.7 Rule)
Assuming a normal distribution:
68% of daily returns fall within ±3.39% of the mean daily return
95% of daily returns fall within ±6.78% (2 × 3.39%) of the mean
99.7% of daily returns fall within ±10.17% (3 × 3.39%) of the mean

3. Annualized Volatility
Your daily volatility of 3.39% translates to approximately 53.8% annualized volatility (3.39% × √252)
This means Bitcoin's price can swing dramatically over a year

4. Risk Assessment
3.39% daily volatility is HIGH for any financial asset
This volatility reflects Bitcoin's speculative nature and market immaturity
It indicates significant price uncertainty and potential for large gains or losses

5. Practical Implications
A "normal" day sees Bitcoin move ±3.39%
An "unusual" day (2 standard deviations) sees moves of ±6.78%
An "extreme" day (3 standard deviations) sees moves of ±10.17%

6. Market Context
This level of volatility is characteristic of:
Cryptocurrency markets (highly speculative)
Emerging markets or new asset classes
Assets with limited liquidity or regulatory uncertainty

In [ ]:
print("rolling volatility: ", get_rolling_volatility(return_series, window = 10))

10-Day Window Interpretation
Time Horizon Context:
Short-term volatility measure - captures recent market behavior
Trading week perspective - roughly 2 weeks of trading activity
Responsive to market changes - quickly reflects shifts in market sentiment
What It Tells You:
Recent Market Stability:
Low rolling volatility = recent price movements have been relatively stable
High rolling volatility = recent price movements have been erratic and unpredictable
Volatility Regime Changes:
Volatility clustering - periods of high volatility tend to cluster together
Regime shifts - sudden changes in rolling volatility indicate market structure changes
Calm before storm - very low rolling volatility often precedes large moves
Trading Implications:
Position sizing - higher rolling volatility suggests smaller position sizes
Stop losses - wider stops needed during high volatility periods
Entry timing - volatility often mean-reverts, so extreme highs/low may signal reversals
Interpreting the Numbers
Typical Bitcoin 10-Day Rolling Volatility:
Low volatility: 1-2% daily (10-20% annualized)
Normal volatility: 2-4% daily (20-40% annualized)
High volatility: 4-6% daily (40-60% annualized)
Extreme volatility: 6%+ daily (60%+ annualized)
Pattern Recognition:
Rising rolling volatility = increasing market uncertainty/risk
Falling rolling volatility = decreasing market uncertainty, potential complacency
Spikes in rolling volatility = market stress events, news shocks, or technical breakdowns

<font color="blue"> HI </font>

In [21]:
def calculate_atr(df: pd.DataFrame, window) -> pd.Series:
    """
    Calculate the Average True Range (ATR) indicator.
    
    ATR measures volatility by calculating the average of the True Range over a specified period.
    True Range is the greatest of:
    - Current High - Current Low
    - |Current High - Previous Close|
    - |Current Low - Previous Close|
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'high', 'low', and 'close' columns
        window (int): Period for calculating the moving average (default: 14)
    
    Returns:
        pd.Series: The Average True Range values
    
    Example:
        atr = calculate_atr(df, window=14)
    """
    # Extract the required price columns
    high = df['high']
    low = df['low']
    close = df['close']
    
    # Calculate the three components of True Range
    # 1. Current High - Current Low
    high_low = high - low
    
    # 2. Absolute value of (Current High - Previous Close)
    high_prev_close = np.abs(high - close.shift(1))
    
    # 3. Absolute value of (Current Low - Previous Close)
    low_prev_close = np.abs(low - close.shift(1))
    
    # True Range is the maximum of the three values
    true_range = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1)
    
    # Calculate ATR using Simple Moving Average
    # Note: Traditional ATR uses Wilder's smoothing (EMA), but SMA is also common
    atr_sma = true_range.rolling(window=window).mean()
    
    return atr_sma

In [15]:
high = df['high']
low = df['low']
close = df['close']
high_low = high - low
    
    # 2. Absolute value of (Current High - Previous Close)
high_prev_close = np.abs(high - close.shift(1))
    
    # 3. Absolute value of (Current Low - Previous Close)
low_prev_close = np.abs(low - close.shift(1))
true_range = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1)

0          0.00
1        827.62
2        805.54
3        692.56
4        814.33
         ...   
2647    1875.00
2648    1361.00
2649    4394.00
2650    3138.00
2651    1192.00
Length: 2652, dtype: float64


In [22]:
print("ATR: ", calculate_atr(df, window = 10))

ATR:  0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
2647    2363.8
2648    2184.3
2649    2482.9
2650    2695.7
2651    2618.6
Length: 2652, dtype: float64


In [17]:
def calculate_volatility_ratio(df: pd.DataFrame, recent_window: int, long_window: int) -> pd.Series:
    """
    Calculate the volatility ratio of a DataFrame.
    
    The volatility ratio is calculated as the ratio of the standard deviation of the recent returns to the standard deviation of the long-term returns.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'close' column
        recent_window (int): Window for recent returns
        long_window (int): Window for long-term returns
    
    Returns:            
        pd.Series: The volatility ratio values
    
    Example:
        volatility_ratio = calculate_volatility_ratio(df, recent_window=20, long_window=50)
    """
    # Calculate recent returns  
    recent_atr = get_rolling_volatility(df, recent_window)
    long_atr = get_rolling_volatility(df, long_window)

    # Calculate volatility ratio
    volatility_ratio = recent_atr / long_atr

    return volatility_ratio

In [18]:
print("Volatility Ratio: ", calculate_volatility_ratio(df, recent_window=20, long_window=50))


Volatility Ratio:  0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
2647    0.874396
2648    0.837973
2649    0.789703
2650    0.805635
2651    0.794770
Length: 2652, dtype: float64


Understanding the Return Value
The ratio tells you:
Ratio > 1.0 = Recent volatility is higher than long-term volatility
Ratio = 1.0 = Recent volatility equals long-term volatility
Ratio < 1.0 = Recent volatility is lower than long-term volatility
Practical Interpretation
High Volatility Ratio (> 1.0):
Market stress - Recent price movements are more volatile than usual
Increased uncertainty - Market participants are more anxious
Potential opportunities - Higher volatility often creates trading opportunities
Risk warning - May indicate market instability or trend changes
Low Volatility Ratio (< 1.0):
Market calm - Recent price movements are less volatile than usual
Complacency - Market participants may be too comfortable
Potential complacency trap - Low volatility often precedes large moves
Range-bound markets - May indicate sideways consolidation

In [40]:
def calculate_bollinger_bands(df: pd.DataFrame, window: int, num_std_dev: int) -> pd.DataFrame:
    """
    Calculate Bollinger Bands for a DataFrame.
    
    Bollinger Bands are a technical analysis tool that consists of three lines:
    - Middle Band: Moving average of the closing prices
    - Upper Band: Middle Band + (num_std_dev * standard deviation of the closing prices)
    - Lower Band: Middle Band - (num_std_dev * standard deviation of the closing prices)

    Parameters:
        df (pd.DataFrame): DataFrame containing 'close' column
        window (int): Window for the moving average
        num_std_dev (int): Number of standard deviations to include in the bands

    Returns:
        pd.DataFrame: DataFrame containing the Bollinger Bands

    Example:
        bollinger_bands = calculate_bollinger_bands(df, window=20, num_std_dev=2)  

    """

    closing_prices = df['close']

    middle_band = closing_prices.rolling(window=window).mean()

    rolling_std = closing_prices.rolling(window=window).std()

    upper_band = middle_band + (num_std_dev * rolling_std)

    lower_band = middle_band - (num_std_dev * rolling_std)

    bollinger_bands = pd.DataFrame({
        'middle_band': middle_band,
        'upper_band': upper_band,
        'lower_band': lower_band
    })

    return bollinger_bands

In [41]:
def calculate_bollinger_bands_width(df: pd.DataFrame, window: int, num_std_dev: int) -> pd.Series:
    """
    Calculate the width of Bollinger Bands.
    
    The width is calculated as the difference between the upper and lower bands divided by the middle band.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'close' column
        window (int): Window for the moving average
        num_std_dev (int): Number of standard deviations to include in the bands

    Returns:
        pd.Series: The width of the Bollinger Bands                                                                 

    Example:
        bollinger_bands_width = calculate_bollinger_bands_width(df, window=20, num_std_dev=2)
    """
    bollinger_bands = calculate_bollinger_bands(df, window, num_std_dev)

    return (bollinger_bands['upper_band'] - bollinger_bands['lower_band']) / bollinger_bands['middle_band']

In [46]:
print("Bollinger Bands Width: ", calculate_bollinger_bands_width(df, window=3, num_std_dev=1))


Bollinger Bands Width:  0            NaN
1            NaN
2       0.142365
3       0.078120
4       0.093647
          ...   
2647    0.018002
2648    0.016208
2649    0.017102
2650    0.013621
2651    0.012642
Length: 2652, dtype: float64


In [20]:
print("Bollinger Bands: ", calculate_bollinger_bands(df, window=20, std_dev=2))


Bollinger Bands:        middle_band    upper_band    lower_band
0             NaN           NaN           NaN
1             NaN           NaN           NaN
2             NaN           NaN           NaN
3             NaN           NaN           NaN
4             NaN           NaN           NaN
...           ...           ...           ...
2647     95904.90  1.506176e+07 -1.486995e+07
2648     96732.00  1.402575e+07 -1.383229e+07
2649     97197.05  1.526176e+07 -1.506737e+07
2650     97726.00  1.685802e+07 -1.666257e+07
2651     98212.05  1.764076e+07 -1.744434e+07

[2652 rows x 3 columns]
